# TFM Máster Data Science UAH 2020-2021

### MIGUEL PÉREZ CARO

Este notebook tiene como objetivo limpiar el dataset para que pueda ser usado posteriormente. Tratamiento del texto:

- Convertir a minúsculas
- Eliminar stopwords
- Eliminar signos de puntuación y símbolos extraños
- Eliminar links
- Tratar los espacios
- Eliminar otro tipo de palabras frecuentes que no aporten valor.

En primer lugar se importan las librerías.

In [1]:
import os
import re
import string
import numpy as np
import pandas as pd
import seaborn as sns
from nltk.corpus import stopwords
from emot.emo_unicode import EMOTICONS_EMO, UNICODE_EMOJI

In [2]:
import warnings
warnings.filterwarnings('ignore')

Se establece el directorio de trabajo.

In [3]:
directorio = os.path.dirname(os.getcwd())
mydir = os.path.join(directorio, 'data')
os.chdir(mydir)

Se carga el archivo

In [4]:
df_raw = pd.read_csv('df_raw.csv')

Se modifica el tipo de los datos

In [5]:
df_raw['id'] = pd.to_numeric(df_raw['id'], errors = 'coerce')
df_raw.dropna(inplace=True)
df_raw['created_at'] = df_raw['created_at'].astype('datetime64[ns]')
df_raw['retweet_count'] = df_raw['retweet_count'].astype('float64')

In [6]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 566643 entries, 0 to 566650
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   id              566643 non-null  float64       
 1   created_at      566643 non-null  datetime64[ns]
 2   text            566643 non-null  object        
 3   retweet_count   566643 non-null  float64       
 4   favorite_count  566643 non-null  float64       
 5   southwest       566643 non-null  float64       
 6   american        566643 non-null  float64       
 7   united          566643 non-null  float64       
 8   delta           566643 non-null  float64       
dtypes: datetime64[ns](1), float64(7), object(1)
memory usage: 43.2+ MB


Se analizan los textos duplicados:

In [7]:
df_raw['text'].duplicated().sum()

239650

Se eliminan los textos duplicados en base a las conclusiones obtenidas en el notebook 03-Analyze-Data. Para ello se hace uso de la función groupby

In [8]:
df_raw_no_dup = df_raw.groupby('text').agg({'id': 'first', 
                                            'created_at':'first', 
                                            'retweet_count':'max',
                                            'favorite_count': 'max',
                                            'southwest': 'max',
                                            'american': 'max',
                                            'united': 'max',
                                            'delta': 'max'}).reset_index()

Se comprueba el resultado

In [9]:
df_raw_no_dup['text'].duplicated().sum()

0

In [10]:
df_raw_no_dup.shape

(326993, 9)

Se convierte el id a un string

In [11]:
df_raw_no_dup['id'] = df_raw_no_dup['id'].astype('object')

In [12]:
df_raw_no_dup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 326993 entries, 0 to 326992
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   text            326993 non-null  object        
 1   id              326993 non-null  object        
 2   created_at      326993 non-null  datetime64[ns]
 3   retweet_count   326993 non-null  float64       
 4   favorite_count  326993 non-null  float64       
 5   southwest       326993 non-null  float64       
 6   american        326993 non-null  float64       
 7   united          326993 non-null  float64       
 8   delta           326993 non-null  float64       
dtypes: datetime64[ns](1), float64(6), object(2)
memory usage: 22.5+ MB


In [13]:
df_raw_no_dup.shape

(326993, 9)

Se genera un csv sin limpiar pero sin texto duplicado.

In [14]:
df_raw_no_dup.to_csv("df_raw_no_dup.csv", sep = ',', index=False, 
              columns = ['id','created_at','text','retweet_count','favorite_count', 
                         'southwest', 'american', 'united', 'delta'])

Para comenzar con la limpieza, se van a generar en primer lugar dos funciones para la limpieza de los emojis y los emoticonos, para continuar con otra función que incluir'a una limpieza más general y llamar'a a estas dos funciones.

In [15]:
def clean_emoticons(text):
    """
    Función generada para hacer la limpieza de emoticonos.
    
    param: text texto a limpiar
    :return: texto limpio
    """
    for emot in EMOTICONS_EMO:
        if emot in text:
            text = text.replace(emot, " {} ".format(EMOTICONS_EMO[emot]))
    return text

In [16]:
def clean_emojis(text):
    """
    Función generada para hacer la limpieza de emojis.
    
    param: text texto a limpiar
    :return: texto limpio
    """
    for emot in UNICODE_EMOJI:
        if emot in text:
            text = text.replace(emot, " {} ".format(UNICODE_EMOJI[emot].replace(':',"").replace('_',' ')))
    return text

Se procede ahora con la limpieza para el análisis de sentimiento. Para ello, se crea una función que realiza lo siguiente:

- Eliminar los usuarios. No aportan información al  análisis de sentimiento que se pretende hacer posteriormente.
- Eliminar los enlaces web.
- Convertir todo el texto a minúsculas.
- Eliminar los emojis y emoticonos. Importante que se eliminen tras la limpieza de los enlaces web ya que estos pueden incluir secuencias de carácteres idénticas a algún emoticono, lo que conllevaría que se preprocesaría como si fuera un emoticono aunque fuera parte de un enlace web.
- Eliminar los signos de almohadilla, ya que el hashtag si que puede contener información. Eliminar la palabra rt, que no aporta valor y aparece con frecuencia. 
- Eliminar los nombres de las aerolíneas que se usaron para la búsqueda ya que tampoco aporta ningún valor al análisis de sentimiento y ya está recogida su presencia en el texto. En estos se incluyen también acrónimos, y las siglas de los números de vuelo. Por ejemplo un vuelo de American Airlines podría ser el AA123, que tras la limpieza resultaría en aa, que no aporta ningún valor.
- Eliminar las abreviaturas usadas para situar a los aviones en vuelo, ya que tampoco aportan valor al texto. En estos casos se incluyen términos como mi, para describir las millas, ft, que son los pies de altura, o frm y hrzn para situar las coordenadas del avión.
- Eliminar lo que no sea minúscula o espacio.
- Normalizar espacios.
- Eliminar stopwords.

In [17]:
lista_elementos_eliminar = ['#', ' rt ', '\n', 'american airlines', 'delta airlines', 'united airlines', 
                            'southwest airlines', ' aa ', ' aal ', ' dl ', ' dal ', ' ua ', ' ual ', ' swa ',
                            ' mi ', ' ft ', ' frm ', ' hrzn ', ' amp ', ' aaa ']

In [18]:
def cleaning_text_basico(df, lista_elementos_eliminar):
    """
    Función generada para hacer una limpieza general del texto.
    
    param: df dataframe a limpiar
    :return: lista con el texto limpiado. Un elemento por cada fila del dataframe.
    """
    corpus = [' ' + text + ' ' for text in  df.text.tolist()] # Añadir espacio al principio y al final apra fcilitar limpieza
    corpus = [re.sub('@[A-Za-z0-9_]+',' ', text) for text in corpus] # Eliminar usuarios
    corpus = [re.sub(r'http\S+', '', text) for text in corpus] # Eliminas enlaces web
    corpus = [text.lower() for text in corpus] # Pasar a minúsculas
    #corpus = [clean_emojis(text) for text in corpus] # Eliminar emojis
    #corpus = [clean_emoticons(text) for text in corpus] # Eliminar emoticonos
    #corpus = [''.join(ch for ch in text if ch not in string.punctuation) for text in corpus] # Eliminar signos de puntuación
    corpus = [re.sub(r'[^a-z\s]', '', text) for text in corpus] # Eliminar lo que no sea minúscula o espacio
    for i in lista_elementos_eliminar:
        corpus = [re.sub(i,' ', text) for text in corpus] # Eliminar elementos de la lista
    corpus = [text.strip() for text in corpus] # Normalizar espacios
    corpus = [re.sub(r' +', ' ', text) for text in corpus] # Normalizar espacios
    corpus = [' '.join(word for word in text.split() if word not in stopwords.words('english')) for text in corpus] # Eliminar stopwords 
    return corpus

In [19]:
corpus_clean = cleaning_text_basico(df_raw_no_dup, lista_elementos_eliminar)

In [20]:
corpus_clean

['cancelled one leg trip eu sent email telling call called multip',
 'im told cannot redeem travel certificate expired middle global pandemi',
 'update still dfw good morning everyone except',
 'attention new million mile flyer alert congrats mr david white achieving one million miles',
 'american delta immediate plans covid vaccine mandates',
 'formerly gonna go well',
 'brands like dennys using college football stars nil era',
 'deltaairlines executives said pair memos employees thursday airlines permanent noflylist',
 'deltaairlines major us airline without vaccinemandate employees',
 'newswire adds daily austin flight expanded service rick husband amarillo international ai',
 'social networking news making sure cant escape tiktok news',
 'social networking news making sure cant escape tiktok news',
 'travel news ramps domestic schedule flights since pandemic began expecting',
 'worldnews story doj challenge jetblue partnership news',
 'worldnews story southwest american rejecting e

Se genera un nuevo dataframe con el contenido limpio

In [21]:
df_clean = df_raw_no_dup.copy()
df_clean['text'] = corpus_clean

Debido a la limpieza, es posible que algunos textos que solo contuvieran un usuario, o el usuario y un enlace web, sean ahora textos vacíos, o solo espacios, por lo que es necesario tratarlo. En primer lugar, se convierten los textos que se compongan únicamente de espacios a valores nulos.

In [22]:
df_clean['text'] = df_clean['text'].replace(r'^\s*$', np.NaN, regex=True)

Se comprueban ahora los valores nulos y se eliminan.

In [23]:
df_clean['text'].isnull().sum()

3949

In [24]:
df_clean.dropna(inplace=True)

Se comprueba si existen textos duplicados.

In [25]:
df_clean['text'].duplicated().sum()

80625

Sí que hay textos duplicados. Esto se puede deber a múltiples factores como por ejemplo:

- Textos cuya única diferencia sea el RT al principio del mismo.
- Textos cuya única diferencia sea la mención a un usuario.
- Textos que contengan links a noticias de forma que el tweet es el mismo pero con un link diferente.

Todo ello puede hacer que tras la limpieza se encuentren nuevamente textos duplicados y que hay que tratar de la misma manera que anteriormente

In [26]:
df_clean_no_dup = df_clean.groupby('text').agg({'id': 'first', 
                                            'created_at':'first', 
                                            'retweet_count':'max',
                                            'favorite_count': 'max',
                                            'southwest': 'max',
                                            'american': 'max',
                                            'united': 'max',
                                            'delta': 'max'}).reset_index()

Se comprueba el nuevo dataframe generado

In [27]:
df_clean_no_dup['text'].duplicated().sum()

0

In [28]:
df_clean_no_dup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242419 entries, 0 to 242418
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   text            242419 non-null  object        
 1   id              242419 non-null  float64       
 2   created_at      242419 non-null  datetime64[ns]
 3   retweet_count   242419 non-null  float64       
 4   favorite_count  242419 non-null  float64       
 5   southwest       242419 non-null  float64       
 6   american        242419 non-null  float64       
 7   united          242419 non-null  float64       
 8   delta           242419 non-null  float64       
dtypes: datetime64[ns](1), float64(7), object(1)
memory usage: 16.6+ MB


In [29]:
df_clean_no_dup.shape

(242419, 9)

A continuación, se muestran por pantalla las palabras más y menos comunes del texto para detectar alguna anomalía que se pueda repetir tanto que sea necesario tratarla.

In [30]:
pd.Series(' '.join(df_clean_no_dup['text']).lower().split()).value_counts()[:50]

flight        39414
get           15318
im            12163
flights       10547
delta         10255
us             9989
time           9250
hours          9219
fly            9161
airlines       9012
dont           8785
like           8562
one            8325
service        7213
airline        6927
people         6842
flying         6780
employees      6731
back           6549
need           6544
customer       6151
would          6108
plane          6045
help           6017
please         5851
travel         5811
got            5810
ive            5653
still          5639
covid          5631
hold           5451
first          5438
know           5298
united         5277
thank          5231
going          5199
never          5152
thanks         5023
new            5013
today          4879
delayed        4831
cant           4813
hour           4662
airport        4621
good           4621
hey            4531
last           4411
vaccinated     4370
go             4275
make           4083


In [31]:
pd.Series(' '.join(df_clean['text']).lower().split()).value_counts()[-50:]

rambles                   1
roadside                  1
thanksforwastingmytime    1
bestplacestolive          1
bestinbiz                 1
wifisucks                 1
corbins                   1
passengerssix             1
sundayampleft             1
nervou                    1
dtr                       1
wordi                     1
maskupforskyjuice         1
gateterminal              1
mixrace                   1
goingi                    1
gracies                   1
msybwi                    1
delayswhat                1
tyran                     1
flightsmsp                1
unpleased                 1
ishouldhavedrove          1
nondire                   1
lgamdw                    1
capitulate                1
maitre                    1
jetwaygeez                1
undermanned               1
hotelresort               1
ims                       1
flyingtocalifornia        1
requesti                  1
mdwcun                    1
hoursdays                 1
listers             

Analizando las palabras más comunes, no hay ningún término que llame la atención, lo cuál podría significar algún tipo de fallo a la hora de preprocesar el texto, por lo que es una buena señal el resultado obtenido. 

En cuanto a las palabras menos comunes, se puede observar que muchas de ellas son fallos a la hora de escribir los vocablos, por lo que podrían ser eliminados, pero no se considera necesario, y se decide no modificar el texto.

In [32]:
df_clean_no_dup.to_csv("df_sentiment_clean_no_dup.csv", sep = ',', index=False, 
              columns = ['id','created_at','text','retweet_count','favorite_count', 
                         'southwest', 'american', 'united', 'delta'])